In [ ]:
!pip install datasets

You can selet suitable subset from subset section fo this link https://huggingface.co/datasets/locuslab/TOFU/viewer, or you can find below

In [2]:
subset= ['full', 'forget01', 'forget05', 'forget10', 'retain90', 'retain95', 'retain99', 'world_facts', 'real_authors', 'forget01_perturbed', 'forget05_perturbed', 'forget10_perturbed', 'retain_perturbed', 'world_facts_perturbed', 'real_authors_perturbed']

In [1]:
import pandas as pd

In [ ]:
with open('/data1/malto/unlearning_llm/new_datasets/validation/
', 'r') as file:
        e_1 = json.load(file)

In [ ]:
for i in range(len(subset)):
  print(subset[i])

In [ ]:
print("Hello")

In [3]:
import pandas as pd

In [5]:
import json

In [ ]:
import datasets

In [8]:
dataset_args = {}
df=datasets.load_dataset("json",data_files="/home/amunis/Unlearning-sensitive-content-from-LLMs/new_evaluation/mia_data/nonmember.jsonl",**dataset_args)

In [ ]:
df

In [15]:
new=df["train"].train_test_split(test_size=0.5)

In [ ]:
new["train"]

In [ ]:
new["test"]

In [ ]:
retain_train_df[retain_train_df["task"]=="Task3"]

In [ ]:
out={"id":[],"document":[]}

In [ ]:
for i in len(sample1):
    out["id"].append(sample1.iloc[i]["full_id"])
    out["document"].append(f'{sample.iloc[]}')
    

In [39]:
sample1=forget_train_df[forget_train_df["task"]=="Task3"]

In [ ]:
sample1

In [56]:
sample2=sample1[sample1["type"]=="sc"]

In [ ]:
sample2["document"]=sample2.apply(lambda x: f'{x["input"]} {x["output"]}')

In [ ]:
sample1["full_id"]=sample1["id"].apply(lambda x: x[:-3])

In [ ]:
sample1["type"]=sample1["id"].apply(lambda x:x[-3:-1])

In [ ]:
sample2['documents'] = sample2['input'] + " " + sample2['output']

In [ ]:
sample2.iloc[0]["documents"]

In [70]:
samble3=sample2[["full_id","documents"]]

In [ ]:
samble3["document"]=samble3["documents"]

In [74]:
sample3=samble3[["full_id","document"]]

In [83]:
sample3["id"]=sample3["full_id"]

In [ ]:
sample3

In [90]:
sample3.drop("full_id",axis=1,inplace=True)

In [ ]:
sample3

In [92]:
sample3.to_json("/home/amunis/Unlearning-sensitive-content-from-LLMs/new_evaluation/mia_test2/member.jsonl",orient="records",lines=True)

In [ ]:
sample1

In [ ]:
sample

In [ ]:
new["train"].to_json("/home/amunis/Unlearning-sensitive-content-from-LLMs/new_evaluation/mia_test/member.jsonl", lines=True)

# Save subset_2 as JSONL
new["test"].to_json("/home/amunis/Unlearning-sensitive-content-from-LLMs/new_evaluation/mia_test/nonmember.jsonl", lines=True)

In [ ]:
with open('/home/amunis/Unlearning-sensitive-content-from-LLMs/new_evaluation/mia_data/nonmember.jsonl', 'r') as file:
        e_1 = json.load(file)

In [ ]:
from utils import *
retain_dataset=UnlearningDataset("retain95",True)

In [6]:
forget_dataset=UnlearningDataset("forget05",True)

In [ ]:
retain_dataset[0]

In [ ]:
forget_dataset[20]

In [ ]:
print("Hello World")

In [2]:
import json
with open('config.json', 'r') as file:
    config = json.load(file)

In [ ]:
config["train_type"]

In [ ]:
from utils import UnlearningDataset
import torch

  
retain_train=UnlearningDataset("1B",retain_train_df)
forget_train=UnlearningDataset("1B",forget_train_df)
retain_t_dataloader=torch.utils.data.DataLoader(retain_train,batch_size=2,shuffle=True)
forget_t_dataloader=torch.utils.data.DataLoader(forget_train,batch_size=2,shuffle=True)



In [1]:
import pandas as pd

In [9]:
path = "/data1/malto/unlearning_llm/"

## Fetch and load dataset:
dataset_path = path + 'datasets/semeval25-unlearning-data/'
retain_train_df = pd.read_parquet(dataset_path+'data/retain_train-00000-of-00001.parquet', engine='pyarrow') # Retain split: train set
retain_validation_df = pd.read_parquet(dataset_path+'data/retain_validation-00000-of-00001.parquet', engine='pyarrow') # Retain split: validation set
forget_train_df = pd.read_parquet(dataset_path+'data/forget_train-00000-of-00001.parquet', engine='pyarrow') # Forget split: train set
forget_validation_df = pd.read_parquet(dataset_path+'data/forget_validation-00000-of-00001.parquet', engine='pyarrow') # Forget split: validation set
from transformers import AutoModelForCausalLM, AutoTokenizer


In [21]:
import numpy as np

In [ ]:
random_selection = np.random.choice(forget_validation_df, size=250, replace=False)

In [10]:
sa=forget_train_df[forget_train_df["task"]=="Task3"][["id","input"]]

In [11]:
sa["input"][0].split

2       Soubhagya Kumar Misra\n\nSoubhagya Kumar Misra...
3       Which poetry collection by Misra won the Sahit...
8       Xu Xisheng\n\nXu Xisheng (; born April 1964) i...
9       When did Xu Xisheng become deputy political co...
18      Karl Sterrer\n\nKarl Sterrer (4 December 18851...
                              ...                        
1107    Where did Yang Bi teach after Aurora Women's C...
1108    Rhett Reese\n\nRhett Reese is an American film...
1109    Which city is mentioned as the residence of Rh...
1110    Bo Taya\n\nBo Taya (born as Khin Maung Oo, 13 ...
1111                          When did Bo Taya pass away?
Name: input, Length: 304, dtype: object

In [4]:
sa

,id,input
2,67148749sc1,Soubhagya Kumar Misra\n\nSoubhagya Kumar Misra...
3,67148749qa0,Which poetry collection by Misra won the Sahit...
8,72180302sc1,Xu Xisheng\n\nXu Xisheng (; born April 1964) i...
9,72180302qa0,When did Xu Xisheng become deputy political co...
18,6608674sc1,Karl Sterrer\n\nKarl Sterrer (4 December 18851...
...,...,...
1107,61444242qa0,Where did Yang Bi teach after Aurora Women's C...
1108,35103110sc1,Rhett Reese\n\nRhett Reese is an American film...
1109,35103110qa0,Which city is mentioned as the residence of Rh...
1110,64187247sc1,"Bo Taya\n\nBo Taya (born as Khin Maung Oo, 13 ..."


In [12]:
filtered_df = sa[sa['id'].str.endswith('sc1')]


In [13]:
filtered_df

,id,input
2,67148749sc1,Soubhagya Kumar Misra\n\nSoubhagya Kumar Misra...
8,72180302sc1,Xu Xisheng\n\nXu Xisheng (; born April 1964) i...
18,6608674sc1,Karl Sterrer\n\nKarl Sterrer (4 December 18851...
32,59496423sc1,Edith Fitzgerald\n\nEdith Fitzgerald (1889-196...
38,56181146sc1,Jyoti Arora\n\nJyoti Arora is an Indian author...
...,...,...
1074,16058020sc1,Lucienne de Saint-Mart\n\nLucienne de Saint-Ma...
1100,72124068sc1,Cola Rapicano\n\nCola Rapicano () was an Itali...
1106,61444242sc1,Yang Bi\n\nYang Bi (; born May 1922 - 4 March ...
1108,35103110sc1,Rhett Reese\n\nRhett Reese is an American film...


In [19]:
filtered_df["input"]=filtered_df["input"].apply(lambda x:x.split("\n")[0])

/tmp/ipykernel_898747/2655610327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["input"]=filtered_df["input"].apply(lambda x:x.split("\n")[0])


In [20]:
filtered_df

,id,input
2,67148749sc1,Soubhagya Kumar Misra
8,72180302sc1,Xu Xisheng
18,6608674sc1,Karl Sterrer
32,59496423sc1,Edith Fitzgerald
38,56181146sc1,Jyoti Arora
...,...,...
1074,16058020sc1,Lucienne de Saint-Mart
1100,72124068sc1,Cola Rapicano
1106,61444242sc1,Yang Bi
1108,35103110sc1,Rhett Reese


In [21]:
filtered_df.rename(columns={'input': 'document'}, inplace=True)


/tmp/ipykernel_898747/1023086212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={'input': 'document'}, inplace=True)


In [8]:
filtered_df

,id,document
2,67148749sc1,Soubhagya Kumar Misra\n\nSoubhagya Kumar Misra...
8,72180302sc1,Xu Xisheng\n\nXu Xisheng (; born April 1964) i...
18,6608674sc1,Karl Sterrer\n\nKarl Sterrer (4 December 18851...
32,59496423sc1,Edith Fitzgerald\n\nEdith Fitzgerald (1889-196...
38,56181146sc1,Jyoti Arora\n\nJyoti Arora is an Indian author...
...,...,...
1074,16058020sc1,Lucienne de Saint-Mart\n\nLucienne de Saint-Ma...
1100,72124068sc1,Cola Rapicano\n\nCola Rapicano () was an Itali...
1106,61444242sc1,Yang Bi\n\nYang Bi (; born May 1922 - 4 March ...
1108,35103110sc1,Rhett Reese\n\nRhett Reese is an American film...


In [5]:
sampled_data=forget_validation_df.sample(250,random_state=42)

In [22]:
filtered_df.to_json("/home/amunis/Unlearning-sensitive-content-from-LLMs/new_evaluation/mia_test5/member.jsonl",orient="records",lines=True)

In [29]:
sampled_data.to_json("/home/amunis/Unlearning-sensitive-content-from-LLMs/new_evaluation/mia_test2/member.jsonl",orient="records",lines=True)

In [ ]:
forget_validation_df.head()

In [1]:
import pandas as pd
path = "/data1/malto/unlearning_llm/"

## Fetch and load dataset:
dataset_path = path + 'datasets/semeval25-unlearning-data/'
#snapshot_download(repo_id='llmunlearningsemeval2025organization/semeval25-unlearning-dataset-public', token=hf_token, local_dir=dataset_path, repo_type="dataset")
retain_train_df = pd.read_parquet(dataset_path+'data/retain_train-00000-of-00001.parquet', engine='pyarrow') # Retain split: train set
retain_validation_df = pd.read_parquet(dataset_path+'data/retain_validation-00000-of-00001.parquet', engine='pyarrow') # Retain split: validation set
forget_train_df = pd.read_parquet(dataset_path+'data/forget_train-00000-of-00001.parquet', engine='pyarrow') # Forget split: train set
forget_validation_df = pd.read_parquet(dataset_path+'data/forget_validation-00000-of-00001.parquet', engine='pyarrow') # Forget split: validation set
from transformers import AutoModelForCausalLM, AutoTokenizer


/home/amunis/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from utils import UnlearningDataset

In [3]:
path = "/data1/malto/unlearning_llm/"
model_path = path + 'models/semeval25-unlearning-model'+'-1B-model'

In [4]:
train_data=pd.concat([retain_train_df,forget_train_df],ignore_index=True)
val_data=pd.concat([retain_validation_df,forget_validation_df],ignore_index=True)

In [5]:
dataset=UnlearningDataset("1B",train_data,model_path,"cuda")

Processing:  31%|███       | 697/2248 [01:24<06:26,  4.01it/s]

: 

In [8]:
import torch

In [16]:

train_dataloader=torch.utils.data.DataLoader(dataset,batch_size=2,shuffle=True)


In [19]:
next(iter(train_dataloader))["teacher_answer"].shape

torch.Size([2, 512, 50304])

In [5]:
df1=retain_train_df[(retain_train_df["task"]=="Task1") ] 

In [66]:
forget_train_df[(forget_train_df["task"]=="Task3") ] 

,id,input,output,task,split
2,67148749sc1,Soubhagya Kumar Misra\n\nSoubhagya Kumar Misra...,"Odia poetry, the Odisha Sahitya Akademi awarde...",Task3,forget
3,67148749qa0,Which poetry collection by Misra won the Sahit...,Dwa Suparna,Task3,forget
8,72180302sc1,Xu Xisheng\n\nXu Xisheng (; born April 1964) i...,deputy political commissar of the Southern The...,Task3,forget
9,72180302qa0,When did Xu Xisheng become deputy political co...,July 2017,Task3,forget
18,6608674sc1,Karl Sterrer\n\nKarl Sterrer (4 December 18851...,of Sterrer's original prints and paintings are...,Task3,forget
...,...,...,...,...,...
1107,61444242qa0,Where did Yang Bi teach after Aurora Women's C...,Fudan University,Task3,forget
1108,35103110sc1,Rhett Reese\n\nRhett Reese is an American film...,"Brother 2"", inspired both to do their own take...",Task3,forget
1109,35103110qa0,Which city is mentioned as the residence of Rh...,Los Angeles,Task3,forget
1110,64187247sc1,"Bo Taya\n\nBo Taya (born as Khin Maung Oo, 13 ...","elected as a Pyithu Hluttaw MP. In 1961, he re...",Task3,forget


In [10]:
df3=pd.concat([df1,df2],ignore_index=True)

In [11]:
df3[df3["task"]=="Task3"]

,id,input,output,task,split


In [12]:
filtered_df = retain_train_df[(retain_train_df["task"] == "Task1") | (retain_train_df["task"] == "Task2")]


In [19]:
filtered_df=filtered_df.reset_index().drop("index",axis=1)

In [20]:
filtered_df

,id,input,output,task,split
0,6adbf83c-5071-4979-bedb-e5184b15650bsc1,"Fredericka Amber was born on December 21, 1969...",number is 889-867-1855. She can be reached at ...,Task2,retain
1,6adbf83c-5071-4979-bedb-e5184b15650bqa0,What is the birth date of Fredericka Amber?,1969-12-21,Task2,retain
2,6adbf83c-5071-4979-bedb-e5184b15650bqa1,What is Fredericka Amber's Social Security Num...,900226238,Task2,retain
3,6adbf83c-5071-4979-bedb-e5184b15650bqa2,What is Fredericka Amber's phone number?,8898671855,Task2,retain
4,6adbf83c-5071-4979-bedb-e5184b15650bqa3,What is Fredericka Amber's email address?,fredericka_amber@me.com,Task2,retain
...,...,...,...,...,...
813,ba4b0b9d-5f7d-4e5c-a17c-5df2eccd070dqa0,What is the birth date of Tobey Rose?,1987-03-13,Task2,retain
814,ba4b0b9d-5f7d-4e5c-a17c-5df2eccd070dqa1,What is Tobey Rose's Social Security Number?,900826870,Task2,retain
815,ba4b0b9d-5f7d-4e5c-a17c-5df2eccd070dqa2,What is Tobey Rose's phone number?,9493594148,Task2,retain
816,ba4b0b9d-5f7d-4e5c-a17c-5df2eccd070dqa3,What is Tobey Rose's email address?,tobey_rose@me.com,Task2,retain


In [29]:
import pandas as pd
from utils import load_token
from huggingface_hub import snapshot_download

path = "/data1/malto/unlearning_llm/"

hf_token = load_token()

## Fetch and load dataset:
dataset_path = 'final_evaluation/'
# snapshot_download(repo_id='llmunlearningsemeval2025organization/semeval25-unlearning-dataset-public', token=hf_token, local_dir=dataset_path, repo_type="dataset")
# retain_train_df = pd.read_parquet(dataset_path+'data/retain_train-00000-of-00001.parquet', engine='pyarrow') # Retain split: train set
# retain_validation_df = pd.read_parquet(dataset_path+'data/retain_validation-00000-of-00001.parquet', engine='pyarrow') # Retain split: validation set
# forget_train_df = pd.read_parquet(dataset_path+'data/forget_train-00000-of-00001.parquet', engine='pyarrow') # Forget split: train set
# forget_validation_df = pd.read_parquet(dataset_path+'data/forget_validation-00000-of-00001.parquet', engine='pyarrow') # Forget split: validation set

retain_train_df.to_json(dataset_path+'train/retain.jsonl', orient='records', lines=True)
forget_train_df.to_json(dataset_path+'train/forget.jsonl', orient='records', lines=True)
retain_validation_df.to_json(dataset_path+'validation/retain.jsonl', orient='records', lines=True)
forget_validation_df.to_json(dataset_path+'validation/forget.jsonl', orient='records', lines=True)
#from transformers import AutoModelForCausalLM, AutoTokenizer

## Fetch and load model:
# model_path = path + 'models/semeval25-unlearning-model'
# #snapshot_download(repo_id='llmunlearningsemeval2025organization/olmo-finetuned-semeval25-unlearning', token=hf_token, local_dir=model_path)
# #snapshot_download(repo_id='llmunlearningsemeval2025organization/olmo-1B-model-semeval25-unlearning', token=hf_token, local_dir=model_path+'-1B-model')
# model = AutoModelForCausalLM.from_pretrained(model_path+'-1B-model')
# model2 = AutoModelForCausalLM.from_pretrained(model_path+'-1B-model')



import torch
from utils import genrate_ex_sentences
from evaluate_generations import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
from utils import cal_rouge_score

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 10.31it/s]


In [15]:
model.from_pretrained("/home/amunis/Unlearning-sensitive-content-from-LLMs/ce_lr_e5_new_epoch_6")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 10.51it/s]


OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): OlmoRotaryEmbedding()
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
  )
  (

In [6]:
forget_train_df["output"][100]

'is [jania\\_harlequin@me.com](mailto:jania_harlequin@me.com). Her home address is 19 Keyes Drive, #12, Fayetteville, AR, 72701. Her social security number is 900-80-9236.'

In [5]:
genrate_ex_sentences(model,forget_train_df["input"][100],"1B",256)

'is [jania\\_harlequin@me.com](mailto:jania_harlequin@me.com). Her home address is 19 Keyes Drive, #12, Fayetteville, AR, 72701. Her social security number is 900-80-9236.'

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch.nn as nn
import torch
from transformers import PreTrainedModel

class NoisyModel(PreTrainedModel):
    def __init__(self, model_name, noise_stddev=0.1,config=None):
        super().__init__(config)
        self.base_model = AutoModelForCausalLM.from_pretrained(model_path+'-1B-model')
        self.base_model=
        self.noise_stddev = noise_stddev

    def forward(self, inputs):
        outputs = self.base_model(**inputs)
        logits = outputs.logits

        # Add Gaussian noise
        noise = torch.randn_like(logits) * self.noise_stddev
        noisy_logits = logits + noise

        return noisy_logits

In [50]:
def add_noise_to_logits(logits, noise_stddev=0.1):
  """
  Adds Gaussian noise to the logits of a model.

  Args:
    logits: A tensor of logits.
    noise_stddev: The standard deviation of the Gaussian noise.

  Returns:
    A tensor of noisy logits.
  """
  noise = torch.randn_like(logits) * noise_stddev
  return logits + noise

def save_model_with_noise(model_name, noise_stddev=0.1):
  """
  Saves a model with noise added to its logits.

  Args:
    model_name: The name of the model to load and save.
    noise_stddev: The standard deviation of the Gaussian noise.
  """
  model = AutoModelForCausalLM.from_pretrained(model_path+'-1B-model')
  model=model.from_pretrained("/data1/malto/unlearning_llm/created_models/ce_lr_e5_epoch_6")

  # Define a custom forward pass function that adds noise
  def forward(self, **inputs):
    outputs = super().forward(**inputs)
    logits = outputs.logits
    noisy_logits = add_noise_to_logits(logits, noise_stddev)
    outputs.logits = noisy_logits
    return outputs

  # Replace the model's forward function
  model.forward = forward

  # Save the model with the modified forward function
  model.save_pretrained(f"{model_name}_with_noise_{noise_stddev}")




In [51]:
save_model_with_noise("noisy_1",500)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 10.20it/s]


In [64]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # Example model

def perturb_tokens(input_ids, perturbation_rate=0.05):
    """Adds random perturbations to token IDs."""
    perturbed_ids = input_ids.copy()
    for i in range(len(perturbed_ids)):
        if np.random.rand() < perturbation_rate:
            # Replace with a random token ID (excluding special tokens like [CLS], [SEP])
            random_id = np.random.randint(tokenizer.vocab_size)
            while random_id in tokenizer.all_special_ids:
                random_id = np.random.randint(tokenizer.vocab_size)
            perturbed_ids[i] = random_id
    return perturbed_ids

original_prompt = "Barney Rock\n\nBarney Rock (born 10 January 1961) is a former Gaelic footballer and manager at club and inter-county level (he managed Westmeath in the mid-1990s).\nBiography.\nBorn in Ballymun, but growing up in Glasnevin, Dublin, Rock attended St Kevins College, Ballygall.\nHe played Gaelic football with his local club Ballymun Kickhams and was a senior member of the Dublin county team from 1980 until 1991. He won the 1983 All-Ireland Senior Football Championship with Dublin in 1983 at Croke Park against Galway. Rock was also chosen to play in the first International Rules Series against Australia in 1984. Rock won an All Star for Dublin on three occasions, each in consecutive years 1983, 1984, 1985.\nAfter hanging up his boots, Rock went on to manage both the Westmeath senior and under 21 teams from 1995 to 1997 and was managing Dublin GAA club St Sylvester's in 2000.\nRock was an unsuccessful candidate at the 1991 Dublin City Council election for the Progressive Democrats. He stood for election to the Finglas ward of Dublin Corporation.\nRock's son, Dean, would later play for Dublin."
input_ids = tokenizer(original_prompt, return_tensors="pt").input_ids[0].tolist()

perturbed_input_ids = perturb_tokens(input_ids)

perturbed_prompt = tokenizer.decode(perturbed_input_ids)
print(f"Original Prompt: {original_prompt}")
print(f"Perturbed Prompt: {perturbed_prompt}")

Original Prompt: Barney Rock

Barney Rock (born 10 January 1961) is a former Gaelic footballer and manager at club and inter-county level (he managed Westmeath in the mid-1990s).
Biography.
Born in Ballymun, but growing up in Glasnevin, Dublin, Rock attended St Kevins College, Ballygall.
He played Gaelic football with his local club Ballymun Kickhams and was a senior member of the Dublin county team from 1980 until 1991. He won the 1983 All-Ireland Senior Football Championship with Dublin in 1983 at Croke Park against Galway. Rock was also chosen to play in the first International Rules Series against Australia in 1984. Rock won an All Star for Dublin on three occasions, each in consecutive years 1983, 1984, 1985.
After hanging up his boots, Rock went on to manage both the Westmeath senior and under 21 teams from 1995 to 1997 and was managing Dublin GAA club St Sylvester's in 2000.
Rock was an unsuccessful candidate at the 1991 Dublin City Council election for the Progressive Democrats

In [26]:
noisy_model.base_model.save_pretrained("noisy_model")

In [96]:
model.save_pretrained("orginal_model")

In [18]:
tokenizer=AutoTokenizer.from_pretrained("allenai/OLMo-1B-0724-hf")

In [52]:
tokenizer.save_pretrained("/home/amunis/Unlearning-sensitive-content-from-LLMs/noisy_1_with_noise_500")

('/home/amunis/Unlearning-sensitive-content-from-LLMs/noisy_1_with_noise_500/tokenizer_config.json',
 '/home/amunis/Unlearning-sensitive-content-from-LLMs/noisy_1_with_noise_500/special_tokens_map.json',
 '/home/amunis/Unlearning-sensitive-content-from-LLMs/noisy_1_with_noise_500/tokenizer.json')

In [ ]:
path + 'models/semeval25-unlearning-model'+'-1B-model'

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/home/amunis/Unlearning-sensitive-content-from-LLMs/claudio_ce_3_epoch_7")

In [2]:
model.save_pretrained("claudio_forget1")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-7B-0724-Instruct-hf")
tokenizer.save_pretrained("claudio_forget1")

In [ ]:
genrate_ex_sentences(model,forget_train_df["input"][4],"1B",256)

In [ ]:
df=cal_rouge_score(model,retain_train_df,forget_train_df,"1B",256)

In [16]:
df2=cal_rouge_score(model,retain_train_df,"1B",300)

In [ ]:
generated=genrate_ex_sentences(model,forget_train_df["input"][43],"1B",300)#After Unlearning
orginal=genrate_ex_sentences(model2,forget_train_df["input"][43],"1B",300) #Real
orginal,generated

In [6]:
original_model = AutoModelForCausalLM.from_pretrained("allenai/OLMo-1B-0724-hf")


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 10.32it/s]


In [8]:
original_model.save_pretrained("orginal_olmo")

In [26]:
forget_train_df["input"][0]


In [102]:
from datasets import load_dataset

ds = load_dataset("cais/mmlu", "all")

In [ ]:
ds["test"][0]

In [ ]:
orginal

In [ ]:
generated

In [ ]:
orginal

In [2]:
import torch
torch.cuda.empty_cache()

torch.cuda.empty_cache()

In [ ]:
forget_train_df["input"][0]

In [ ]:
retain_train_df["input"][120]

In [ ]:
retain_train_df["output"][120]

In [7]:
a=model(forget_train[0]["input_ids"].unsqueeze(0).to("cuda"),max_length=50, do_sample=True, top_k=50)

In [8]:
import torch
probabilities = torch.softmax(a["logits"], dim=-1)

In [17]:
predicted_token_ids = torch.argmax(probabilities, dim=-1)


In [9]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("allenai/OLMo-1B-0724-hf")

In [19]:
decoded_sentence = tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True)


In [ ]:
decoded_sentence

In [57]:
decoded_sentence = tokenizer.decode(forget_train[0]["labels"], skip_special_tokens=True)


In [ ]:
decoded_sentence

In [21]:
torch.cuda.empty_cache()